In [74]:
import pandas as pd
import numpy as np
import math
import csv
import re
import json
import time
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions 
from pyspark.sql import Row
from collections import OrderedDict


In [75]:
# parsing data from sgm file

from bs4 import BeautifulSoup,SoupStrainer


def readData(fileName):
    
    data = ''
    f = open(fileName, 'rb')
    soup = BeautifulSoup(f, 'html.parser')
    # get body content
    contents = soup.findAll('body')
    
    return contents

# parsing data from sgm file

In [76]:
# Q1

# ==== initial setting ====
sTime = time.time()
file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]
bodyLen, shingleLen = 0, 0
# storage for term
wordSet = set()
# storage for term Frequency
wordList = []
# ==== initial setting ====

# process all data set
print("Start count word Frequency")
for i in range(len(file)):
    fileName = "/home/ethan/pythonwork/ipynotebook/HW4/Data/reut2-"
    fileName = fileName + file[i] + ".sgm"
    # read body content from each file
    data = readData(fileName)
    # retrival all body content
    for content in data:
        if i < 2:
            bodyLen +=1
        # split one body content by \n
        for d in content.get_text().split("\n"):
            # remove header space and footer space
            sentence = d.strip()
            # regular expresisson
            pat = '[a-zA-Z]+'
            reContent = re.findall(pat, sentence)
            # select one word on each time
            for ind in range(len(reContent)):
                # add word
                wordSet.add(reContent[ind])
    print("file ", i, " done")

# for speed up word count matrix
for w in wordSet:
    wordList.append(w)
wordList.sort()
# release memory
del wordSet
wordLen = len(wordList)

print("Count word Frequency done\n")
print("Total body number: %d" % bodyLen)
print("Total word number: %d" % wordLen)
print("Count word Frequency took: %.2fs" % (time.time()-sTime))

# Q1


Start count word Frequency
file  0  done
file  1  done
file  2  done
file  3  done
file  4  done
file  5  done
file  6  done
file  7  done
file  8  done
file  9  done
file  10  done
file  11  done
file  12  done
file  13  done
file  14  done
file  15  done
file  16  done
file  17  done
file  18  done
file  19  done
file  20  done
file  21  done
Count word Frequency done

Total body number: 1855
Total word number: 45595
Count word Frequency took: 41.23s


In [77]:
# Q1

# ==== initial setting ====

sTime = time.time()
file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]
matchNum = 0
col = 0
pat = '[a-zA-Z]+'
matrix = np.zeros((wordLen,bodyLen), int)
totalWord = 0
# ==== initial setting ====

# process specify document
print("Start calculate word Frequency matrix")
for i in range(2):
    fileName = "/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-"
    fileName = fileName + file[i] + ".sgm"
    print(fileName +" start!")
    # read body content from each file
    data = readData(fileName)
    contInd = 0
    for content in data:
        # save total term frequency on one body content
        bContWordDict = {}
        # split one body content by \n
        for d in content.get_text().split("\n"):
            # remove header space and footer space
            sentence = d.strip()
            # regular expresisson
            reContent = re.findall(pat, sentence)
            # select one word on each time
            for r in range(len(reContent)):
                if reContent[r] not in bContWordDict:
                    bContWordDict[reContent[r]] = 1
                else:
                    bContWordDict[reContent[r]] +=1
        # sort result in alphabetical order
        orderRes = OrderedDict(sorted(bContWordDict.items(), key=lambda x: x[0]))
        totalWord += len(orderRes)
        # process all term frequency on each body content
        row = 0
        for w in range(wordLen):
            try:
                wordWithCount = list(orderRes.items())[row]
            except IndexError:
                print(row)
            # find out term appeared location, then append value to matrix
            if wordWithCount[0] == wordList[w]:
                matrix[w][col] = wordWithCount[1]
                matchNum += 1
                row +=1
            # when visit all term on one body content then break this loop
            if row == len(orderRes):
                break
        col +=1
        contInd +=1
        if contInd %100 == 0:
            print(contInd, " done!")
        
print("Calculate word Frequency matrix done")             
print("total Word num %d" % totalWord)
print("total match num %d" % matchNum)
print("Calculate word Frequency matrix took %.2fs" % (time.time()-sTime))

# save Q1 Result
sTime = time.time()
np.savetxt("Q1_Result.csv", matrix, delimiter=",")
print("Save word Frequency matrix took %.2f" % (time.time()-sTime))



# Q1

Start calculate word Frequency matrix
/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-000.sgm start!
100  done!
200  done!
300  done!
400  done!
500  done!
600  done!
700  done!
800  done!
900  done!
/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-001.sgm start!
100  done!
200  done!
300  done!
400  done!
500  done!
600  done!
700  done!
800  done!
900  done!
Calculate word Frequency matrix done
total Word num 138615
total match num 138615
Calculate word Frequency matrix took 306.47s
Save word Frequency matrix took 78.16


In [78]:

# # Q2

# # ==== initial setting ====

# sTime = time.time()
# index = 0
# flag = 0
# minHash = np.zeros(bodyLen,int)
# idx = np.full(shingleLen, False, dtype=bool)
# curNum = 0
# ind = 0
# mod = 10

# # ==== initial setting ====

# print("Start process minHash!")
# while index < shingleLen and flag < bodyLen:
#     curNum = ind
#     index +=1
#     if(curNum + mod)< shingleLen:
#         ind = curNum + mod
#     else:
#         ind = (curNum + mod) % shingleLen
#     while idx[ind] == True:
#         ind += 1
#     idx[ind] = True
#     for c in range(bodyLen):
#         if matrix[ind][c] == True:
#             if minHash[c] == 0:
#                 minHash[c] = index
#                 flag +=1
#     if index % 2000 ==0:
#         print("flag ",flag, " done!")
#         print("index ", index, " done!")
        
# print("Process minHash done!")
# print("MinHash calculate took %.2f" % (time.time()-sTime))

# # save Q2 Result
# sTime = time.time()
# np.savetxt("Q2_Result.csv", minHash, delimiter=",")
# print("Save minHash matrix took %.2fs" % (time.time()-sTime))


# # Q2


In [79]:
# from collections import OrderedDict

# s = set()

# s.add("1")
# s.add("2")
# s.add("3")
# s.add("b")
# s.add("ac")
# s.add("ed")
# s.add("a")
# #print(s)

# l = []
# for i in s:
#     l.append(i)   
# l.sort()

# #print(l)


# d = {}
# d["b"] = 123
# d["ac"] = 45
# d["ed"] = 66
# d["a"] = 777


# ans = OrderedDict(sorted(d.items(), key=lambda x: x[0]))


# print(ans)
# print(l)

# row = 0
# for i in l:
	
# 	asd = list(ans.items())[row]
# 	print(asd[0]," ", i)
	
# 	if asd[0] == i:
# 		print(i)
# 		row+=1
# 	print("row ", row)
# 	if row == len(ans)-1:
# 		print("HI")
# 		break





